In [2]:
from contextlib import redirect_stdout
import discord
import random
import os
import asyncio

import pandas as pd
import requests

#########################

import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



#match_id = match_id

In [26]:
df_all= pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_03/df_all_round_sim.csv')



/Users/roumyadas/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
match_id = 'S02M074'

In [25]:
def match_awards(df_all, match_id):

    output_file = "/Users/roumyadas/Desktop/IPL_Simulation/Season_03/awards/awards_" + str(match_id) + ".txt"
    #output_file = "/Users/roumyadas/Desktop/IPL_Simulation/experimentation/matchcard_" + str(match_id) + ".txt"

    m_id = int(match_id.replace('S02M',''))
    # Redirect the output
    with open(output_file, "w") as f:
        with redirect_stdout(f):

            ## highest SR (runs) [10 balls], 4s (runs), 6s (runs), most dots (eco)

            sub_df = df_all[(df_all.match_id==match_id)]


            ### most dots
            bowler_stats = sub_df.groupby(['bowler','bowling_team']).agg(   ##,'innings'
                runs = ('runs_conceeded','sum'),
                balls = ('islegal' ,'sum'),
                wkts = ('isBowlerWicket','sum'),
                dots = ('isDotforbowler','sum')
            ).reset_index()

            bowler_stats = bowler_stats[bowler_stats.balls>=1]
            bowler_stats['economy'] = 6*bowler_stats['runs']/bowler_stats['balls']
            bowler_stats = bowler_stats.sort_values(['dots','economy','wkts','balls'], 
                                                    ascending=[False, True, False, True]).reset_index(drop=True)
            bowler_stats['name'] = bowler_stats[['bowler','bowling_team','dots']]\
                                                .apply(lambda x: f"{x[0]} ({x[1]}) : {x[2]} dots", axis=1)

            value = bowler_stats.head(1)['name'].unique()[0]

            print(f"AWARDS ::: \n")
            print(f"Most Dots --> {value}")

            ### highest SR

            batter_stats = sub_df.groupby(['striker','batting_team']).agg(  ##,'innings'
                runs = ('runs_off_bat','sum'),
                balls = ('is_faced_by_batter' ,'sum'),
                fours = ('isFour','sum'),
                sixes = ('isSix','sum'),
                outs = ('is_striker_Out','sum'),
                dots = ('isDotforBatter','sum')

            ).reset_index()

            batter_stats = batter_stats[batter_stats.balls>=10]
            batter_stats['strike_rate'] = 100*batter_stats['runs']/batter_stats['balls']
            batter_stats = batter_stats.sort_values(['strike_rate','runs','balls','dots'], 
                                                    ascending=[False, False, True, True]).reset_index(drop=True)
            batter_stats['name'] = batter_stats[['striker','batting_team','strike_rate']]\
                                                .apply(lambda x: f"{x[0]} ({x[1]}) : {np.round(x[2],2)} SR", axis=1)

            value = batter_stats.head(1)['name'].unique()[0]

            print(f"Highest Strike-rate --> {value}")

            ### most 4s

            batter_stats = sub_df.groupby(['striker','batting_team']).agg(  ##,'innings'
                runs = ('runs_off_bat','sum'),
                balls = ('is_faced_by_batter' ,'sum'),
                fours = ('isFour','sum'),
                sixes = ('isSix','sum'),
                outs = ('is_striker_Out','sum'),
                dots = ('isDotforBatter','sum')

            ).reset_index()

            batter_stats = batter_stats[batter_stats.fours>=1]
            batter_stats = batter_stats.sort_values(['fours','runs','balls','sixes'], 
                                                    ascending=[False, False, True, False]).reset_index(drop=True)
            batter_stats['name'] = batter_stats[['striker','batting_team','fours']]\
                                                .apply(lambda x: f"{x[0]} ({x[1]}) : {np.round(x[2],2)} fours", axis=1)

            value = batter_stats.head(1)['name'].unique()[0]

            print(f"Most Fours --> {value}")

            ### most 6s

            batter_stats = sub_df.groupby(['striker','batting_team']).agg(  ##,'innings'
                runs = ('runs_off_bat','sum'),
                balls = ('is_faced_by_batter' ,'sum'),
                fours = ('isFour','sum'),
                sixes = ('isSix','sum'),
                outs = ('is_striker_Out','sum'),
                dots = ('isDotforBatter','sum')

            ).reset_index()

            batter_stats = batter_stats[batter_stats.sixes>=1]
            batter_stats = batter_stats.sort_values(['sixes','runs','balls','fours'], 
                                                    ascending=[False, False, True, False]).reset_index(drop=True)
            batter_stats['name'] = batter_stats[['striker','batting_team','sixes']]\
                                                .apply(lambda x: f"{x[0]} ({x[1]}) : {np.round(x[2],2)} sixes", axis=1)

            value = batter_stats.head(1)['name'].unique()[0]

            print(f"Most Sixes --> {value}")

    message = 'awards writing done'
    return(message)

In [28]:
for match_id in df_all['match_id'].unique():
    match_awards(df_all, match_id)